# Cleveland Heart Disease Binary Classification

# Introduction

One of the leading causes of death within Canada and globally is heart disease (Finegold et al., 2013). Heart disease comprises a range of multifaceted conditions including heart failure and artery blockage (Sun et al., 2015). These conditions can be influenced by a myriad of factors such as genetics and lifestyle (Blue et al., 2012). In this project, we used binary classification (present or absent) to answer the question: What is the risk of heart disease based on the analysis of three key attributes - maximum heart rate achieved, age, and ST-wave depression induced by exercise?

We chose these three variables for the following reasons. Higher maximum heart rates could possibly indicate better cardiovascular fitness. Age is a well-studied risk factor for heart disease; showing that the risk of heart disease occurrence increases with age. Lastly, the degree of ST-wave depression induced by exercise relative to rest is associated with a greater likelihood of heart disease as it reaches higher levels. 

By investigating the relationship between these variables and the presence of heart disease, we created a predictive model that can effectively identify individuals at risk of heart disease. This classification was conducted on the Cleveland heart disease data set which contains 303 entries across 14 different variables.

# Methods

In [10]:
library(tidyverse)
library(repr)
library(readxl)
library(tidymodels)
library(dplyr)
library(kknn)
library(ggplot2)
library(cowplot)

ERROR: Error in library(kknn): there is no package called ‘kknn’


Cleveland data set on heart disease was retrieved from the UCI machine learning repository. Data set was cleaned and wrangled to ensure the data is in a suitable format for further processing. Column 14 (Target) was renamed to “Diseased” to clarify if heart disease is present or absent in the diagonsis.

In [5]:
#Reading dataset, includes column renaming
set.seed(29)
file_path <- "https://github.com/colemantatiana/dsci-100-2023w1-group-29/raw/main/heart.csv"
heart_data <- read_delim(file_path, col_names = TRUE)

#Conversion of "target" to factor
heart_data <- mutate(heart_data, target = as.factor(target))

#Renaming
colnames(heart_data)[14] = "diseased"
    heart_data |> distinct(diseased)

# Check for missing values.
missing <- heart_data |> 
  summarise(across(everything(), ~ sum(is.na(.x))))
missing 
#No missing data 

# Overview of the data
glimpse(heart_data)
summary(heart_data)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


diseased
<fct>
1
0


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diseased
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,0,0,0,0,0


Rows: 303
Columns: 14
$ age      <dbl> 63, 37, 41, 56, 57, 57, 56, 44, 52, 57, 54, 48, 49, 64, 58, 5…
$ sex      <dbl> 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1…
$ cp       <dbl> 3, 2, 1, 1, 0, 0, 1, 1, 2, 2, 0, 2, 1, 3, 3, 2, 2, 3, 0, 3, 0…
$ trestbps <dbl> 145, 130, 130, 120, 120, 140, 140, 120, 172, 150, 140, 130, 1…
$ chol     <dbl> 233, 250, 204, 236, 354, 192, 294, 263, 199, 168, 239, 275, 2…
$ fbs      <dbl> 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0…
$ restecg  <dbl> 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1…
$ thalach  <dbl> 150, 187, 172, 178, 163, 148, 153, 173, 162, 174, 160, 139, 1…
$ exang    <dbl> 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0…
$ oldpeak  <dbl> 2.3, 3.5, 1.4, 0.8, 0.6, 0.4, 1.3, 0.0, 0.5, 1.6, 1.2, 0.2, 0…
$ slope    <dbl> 0, 0, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 0, 2, 2, 1…
$ ca       <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0…
$ thal     <dbl> 1

      age             sex               cp           trestbps    
 Min.   :29.00   Min.   :0.0000   Min.   :0.000   Min.   : 94.0  
 1st Qu.:47.50   1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:120.0  
 Median :55.00   Median :1.0000   Median :1.000   Median :130.0  
 Mean   :54.37   Mean   :0.6832   Mean   :0.967   Mean   :131.6  
 3rd Qu.:61.00   3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:140.0  
 Max.   :77.00   Max.   :1.0000   Max.   :3.000   Max.   :200.0  
      chol            fbs            restecg          thalach     
 Min.   :126.0   Min.   :0.0000   Min.   :0.0000   Min.   : 71.0  
 1st Qu.:211.0   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:133.5  
 Median :240.0   Median :0.0000   Median :1.0000   Median :153.0  
 Mean   :246.3   Mean   :0.1485   Mean   :0.5281   Mean   :149.6  
 3rd Qu.:274.5   3rd Qu.:0.0000   3rd Qu.:1.0000   3rd Qu.:166.0  
 Max.   :564.0   Max.   :1.0000   Max.   :2.0000   Max.   :202.0  
     exang           oldpeak         slope             ca        
 Mi

Forward selection is performed below. Data was split into a training set and test set, using only the training set to pick the predictors. From the training data, we identified three relevant predictors for our analysis: (1) thalach (maximum heart rate achieved) (2) age, and (3) old peak (ST-wave depression induced by exercise). 

In [14]:
# Create training (75%) and testing (25%) data split 
heart_split <- initial_split(heart_data, prop = 0.75, strata = diseased)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)

# View training and testing sets
glimpse(heart_train)
glimpse(heart_test)

#Variable selection was performed on five relevant variables 

names <- colnames(heart_train |> select(age,trestbps, chol, thalach, oldpeak))

accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

#Cross validation to generate model accuracy 
heart_vfold <- vfold_cv(heart_train, v = 5, strata = diseased)


n_total <- length(names)
selected <- c()
for (i in 1:n_total) {
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        preds_new <- c(selected, names[[j]])
        model_string <- paste("diseased", "~", paste(preds_new, collapse="+"))
        heart_recipe <- recipe(as.formula(model_string), 
                                data = heart_train) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())
        acc <- workflow() |>
          add_recipe(heart_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = heart_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
accuracies
#Three variables had highest accuracy, so therefore we selected. 

Rows: 226
Columns: 14
$ age      <dbl> 67, 67, 53, 58, 60, 60, 64, 43, 57, 55, 65, 50, 44, 60, 54, 5…
$ sex      <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1…
$ cp       <dbl> 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0…
$ trestbps <dbl> 160, 120, 140, 132, 130, 117, 140, 120, 150, 132, 150, 150, 1…
$ chol     <dbl> 286, 229, 203, 224, 206, 230, 335, 177, 276, 353, 225, 243, 2…
$ fbs      <dbl> 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ restecg  <dbl> 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0…
$ thalach  <dbl> 108, 129, 155, 173, 132, 160, 158, 120, 112, 132, 114, 128, 1…
$ exang    <dbl> 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1…
$ oldpeak  <dbl> 1.5, 2.6, 3.1, 3.2, 2.4, 1.4, 0.0, 2.5, 0.6, 1.2, 1.0, 2.6, 0…
$ slope    <dbl> 1, 1, 0, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0…
$ ca       <dbl> 3, 2, 0, 2, 2, 2, 0, 0, 1, 1, 3, 0, 1, 1, 1, 1, 0, 3, 2, 0, 0…
$ thal     <dbl> 2

ERROR: [1m[33mError[39m in `check_installs()`:[22m
[33m![39m Some package installs are required: 
[36m•[39m 'kknn', 'kknn'


For our initial visualization (Figure 1), we used 2 scatter plots to show two of our predictor variables as a function of the third: age. 

In [13]:
#Initial visualization was conducted 
heart_plot1 <- heart_train |>
  ggplot(aes(x = age, 
             y = thalach, 
             colour = diseased)) +
  labs(x = "Age (years)", y = "Maximum Heart Rate Achieved (beats per minute)") +
  theme(legend.position = "none") +
  geom_point() +
  ggtitle("Visualization of Predictor Variables: Age and Thalach") +
  scale_color_manual(values = c("#f8766d", "#00b0f6"))

heart_plot2 <- heart_train |> ggplot(aes(x = age, y = oldpeak, colour = diseased)) + 
    geom_point() + 
    ggtitle("Age and Oldpeak") + 
    labs(x = "Age (years)", y = "ST Depression Induced by Exercise (relative to rest)", colour = "Diagnosis") +
    scale_color_manual(labels = c("Diseased", "Healthy"), values = c("#f8766d", "#00b0f6"))

options(repr.plot.height = 7, repr.plot.width = 12)
heart_plots <- plot_grid(heart_plot1, heart_plot2,
                         ncol = 2, nrow = 1, 
                         rel_widths = c(4,5))
heart_plots

ERROR: Error in plot_grid(heart_plot1, heart_plot2, ncol = 2, nrow = 1, rel_widths = c(4, : could not find function "plot_grid"


**Figure 1. Scatter plots to visualize our predictor variables: Age and Thalach, Age and Old peak**

Next, we choose the number of neighbors, K, for the model with our three variables (risk factors). The best number of neighbors was decided by using cross-validation to calculate an accuracy for each value of K, and then the value of K with the best accuracy was picked. Accuracy vs K was plotted as shown in Figure 2. below. K = 21 provides the highest accuracy and was therefore selected.

In [18]:
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

knn_results <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = heart_vfold, grid = k_vals) |>
  collect_metrics() 

k_accuracies <- knn_results |>
  filter(.metric == "accuracy")

#Accuracy vs K plot generated
accuracy_vs_k <- ggplot(k_accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") + 
  theme(text = element_text(size = 12))

accuracy_vs_k

ERROR: [1m[33mError[39m in `check_installs()`:[22m
[33m![39m Some package installs are required: 
[36m•[39m 'kknn', 'kknn'


**Figure 2. Number of Neighbors vs Accuracy Estimate**

We can now build our final model using k = 21. 

In [19]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 21) |>
  set_engine("kknn") |>
  set_mode("classification")

heart_recipe <- recipe(diseased ~ thalach+oldpeak+age, data = heart_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_fit <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
  fit(data = heart_train)

knn_fit

ERROR: [1m[33mError[39m in `check_installs()`:[22m
[33m![39m This engine requires some package installs: 'kknn'


The performance of our model was evaluated on the test data set. Here we predict the labels in the test set and assess the classifier’s performance by examining accuracy (using the metrics function). Looking at the .estimate variable, the estimated accuracy of the classifier on the test data was 73%. 

In [17]:
heart_test_predictions <- predict(knn_fit, heart_test) |>
  bind_cols(heart_test)
heart_test_predictions

ERROR: Error in predict(knn_fit, heart_test): object 'knn_fit' not found


In [9]:
heart_test_predictions |>
  metrics(truth = diseased, estimate = .pred_class) |>
  filter(.metric == "accuracy")
heart_test_predictions

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7272727


.pred_class,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diseased
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
1,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
1,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
1,49,1,1,130,266,0,1,171,0,0.6,2,0,2,1
1,44,1,2,130,233,0,1,179,1,0.4,2,0,2,1
1,61,1,2,150,243,1,1,137,1,1.0,1,0,2,1
1,53,1,2,130,197,1,0,152,0,1.2,0,0,2,1
1,39,1,2,140,321,0,0,182,0,0.0,2,0,2,1
1,44,1,2,140,235,0,0,180,0,0.0,2,0,2,1


Confusion matrix was generated to calculate precision and recall. Recall is 92.86%. Precision is 68.42%. 

In [10]:
confusion <- heart_test_predictions |>
             conf_mat(truth = diseased, estimate = .pred_class)
confusion

          Truth
Prediction  0  1
         0 17  3
         1 18 39

The confusion matrix for the classifier was then analyzed. The matrix shows 17 observations correctly classified as absent, and 39 were correctly classified as present. Some classifier mistakes are shown, classifying 18 observations as present when truly absent and 3 observations as absent when truly present. 

(1 = present 0 = absent)

In [ ]:
heart_proportions <- heart_train |>
                      group_by(diseased) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(heart_train))
heart_proportions

# References

Blue, G. M., Kirk, E. P., Sholler, G. F., Harvey, R. P., & Winlaw, D. S. (2012). Congenital heart disease: Current knowledge about causes and inheritance. *Medical Journal of Australia, 197*(3). [Read Article](https://www.mja.com.au/journal/2012/197/3/congenital-heart-disease-current-knowledge-about-causes-and-inheritance)

Finegold, J. A., Asaria, P., & Francis, D. P. (2013). Mortality from ischaemic heart disease by country, region, and age: Statistics from World Health Organisation and United Nations. *International Journal of Cardiology, 168*(2), 934–945. [DOI: 10.1016/j.ijcard.2012.10.046](https://doi.org/10.1016/j.ijcard.2012.10.046)

Sun, R., Liu, M., Lu, L., Zheng, Y., & Zhang, P. (2015). Congenital Heart Disease: Causes, Diagnosis, Symptoms, and Treatments. *Cell Biochemistry and Biophysics, 72*(3), 857–860. [DOI: 10.1007/s12013-015-0551-6](https://doi.org/10.1007/s12013-015-0551-6)

Peng, T. T., Trevor Campbell, and Melissa Lee Foreword by Roger. (2023). *Data Science*. In [datasciencebook.ca](https://datasciencebook.ca/). [CC BY-NC-SA 4.0 License](https://creativecommons.org/licenses/by-nc-sa/4.0/). [Read Online](https://datasciencebook.ca/index.html)

UCI Machine Learning Repository. (n.d.). [Archive.ics.uci.edu](https://archive.ics.uci.edu/dataset/45/heart+disease).
